[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

***

# Tutorial \#3: Hands-on Scientific Computing with LibKet - Part 2

> In this tutorial you will learn to
> 1. write quantum expressions with for-loops
> 2. define custom quantum gates
> 3. build quantum algorithms from components

## Getting started
Let's include **LibKet**'s main headerfile, import its namespaces, and inject the code for displaying images. This can take some time, stay tuned.

In [70]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

A common building block in many quantum algorithms is the [Quantum Fourier Transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) (QFT) and its inverse QFT$^\dagger$

![QFT](../images/qft_circuit.png)

where **CRk** denotes the **controlled phase shift** gate, ``crk<k>(ctrl, target)``, with angle $\theta = \pi/2^k$ (radians).

**LibKet** also has a **CR($\theta$)** gate that supports arbitrary rotation angles $\theta$, which have to be passed as **compile-time constants** ``QConst(value)`` in the first argument, i.e. ``cr(QConst(1.5708), ctrl, target)``.

The **CRk** gate is particularly useful for implementing the QFT as the rotation angles vary with the distance between the control and the target qubit.

**LibKet** has ready-to-use quantum expressions ``qft(...)`` and ``qftdag(...)``. Let's start with the former one.

In [72]:
auto expr = LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(init());

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cu1(1.570796326794896558) q[1], q[0];
cu1(0.785398163397448279) q[2], q[0];
cu1(0.392699081698724139) q[3], q[0];
h q[1];
cu1(1.570796326794896558) q[2], q[1];
cu1(0.785398163397448279) q[3], q[1];
h q[2];
cu1(1.570796326794896558) q[3], q[2];
h q[3];



Don't be surprised to see ``cu1(theta) q[i], q[j]`` instead of ``crk``. This is specific [OpenQASM](https://en.wikipedia.org/wiki/OpenQASM) code, one of the many quantum assembly languages, and it does not support the **CRk** gate natively. **LibKet** frees us from these technical subleties and allows us to write backend-agnostic code.

The pre-implemented ``qft<mode>(...)`` function makes use of another advanced **LibKet** feature.

## Compile-time for loops with ``static_for()``

The generic interface of the ``static_for()`` function reads
```cpp
template<index_t for_start,
         index_t for_end,
         index_t for_step,
         template<index_t start, index_t end, index_t step, index_t index>
         class functor,
         typename functor_return_type,
         typename... functor_types>
inline auto
static_for(functor_return_type&& functor_return_arg,
           functor_types&&... functor_args)
```
Let's start with a small
####  Example
We first create a **functor**, which represents the loop's body

In [9]:
template<index_t start, index_t end, index_t step, index_t index>
struct ftor 
{
    template<typename  Expr>
    inline constexpr auto operator()(Expr&& expr) noexcept 
    {
        // Returns the controlled phase shift gate with angle
        // theta = pi/2^(index+1) between qubits index and index+1
        return crk<index+1>(sel<index>  (gototag<0>()),
                            sel<index+1>(gototag<0>(expr))
                           );
    }
};

To *loop* through this functor (at compile time) we call the ``utils::static_for<start, end, step, body>(...)`` function as follows. Note the usefulness of the ``tag``/``gototag`` mechanism to restore the original filter settings easily

In [11]:
auto expr = utils::static_for<0,4,1,ftor>(tag<0>(init()));

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
cu1(3.141592653589793116) q[0], q[1];
cu1(1.570796326794896558) q[1], q[2];
cu1(0.785398163397448279) q[2], q[3];
cu1(0.392699081698724139) q[3], q[4];
cu1(0.196349540849362070) q[4], q[5];



#### Nested loops
Compile-time for loops can even be nested as it is done in our QFT implementation. Here the outer loop repeatedly calls the inner loop with different values of the ``start`` index

In [12]:
template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_inner
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return crk<index-start+2>(sel<index  >(gototag<0>(expr0)),
                                  sel<start-1>(gototag<0>(expr1))
                                 );
    }
};

template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_outer
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return utils::static_for<index+1, end, 1, qft_loop_inner>
            (h(sel<index>(gototag<0>(expr0))),
             gototag<0>(expr1)
            );
    }
};

Armed with the nested compile-time for-loop we can implement the QFT in a simple one-liner.

In [16]:
auto expr = utils::static_for<0,3,1,qft_loop_outer>(tag<0>(init()), tag<0>());

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cu1(1.570796326794896558) q[1], q[0];
cu1(0.785398163397448279) q[2], q[0];
cu1(0.392699081698724139) q[3], q[0];
h q[1];
cu1(1.570796326794896558) q[2], q[1];
cu1(0.785398163397448279) q[3], q[1];
h q[2];
cu1(1.570796326794896558) q[3], q[2];
h q[3];



What happens if you move the ``init()`` from the first to the second argument?

#### Execise: ``allswap()``
Now it's your turn. Write a functor and a compile-time for-loop that ``swap``s $\lvert q_0\rangle$ with $\lvert q_n\rangle$, $\lvert q_1\rangle$ with $\lvert q_{n-1}\rangle$, etcetera. 

Apply your implementation *after* the QFT and compare the generated OpenQASM code with out reference implementation
```cpp
auto expr = LibKet::circuits::qft(...);
```
or
```cpp
auto expr = LibKet::circuits::allswap(LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(...));
```

In [58]:
template<index_t start, index_t end, index_t step, index_t index>
struct add_loop_inner
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return crk<index+1>(sel<index>(gototag<0>(expr0)),
                            gototag<1>(expr1)
                           );
    }
};

template<index_t start, index_t end, index_t step, index_t index>
struct add_loop_outer
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return utils::static_for<index, end/2, 1, add_loop_inner>
            (sel<index>(gototag<0>(expr0)), 
             tag<1>(sel<(end+1)/2+index>(gototag<0>(expr1)))
            );
    }
};

In [59]:
auto expr = utils::static_for<0, 5, 1, add_loop_outer>(tag<0>(init()), tag<0>());

QDevice<QDeviceType::qiskit_qasm_simulator, 6> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
creg c[6];
cu1(3.141592653589793116) q[0], q[3];
cu1(1.570796326794896558) q[1], q[3];
cu1(0.785398163397448279) q[2], q[3];
cu1(1.570796326794896558) q[1], q[4];
cu1(0.785398163397448279) q[2], q[4];
cu1(0.785398163397448279) q[2], q[5];



In [80]:
auto expr = LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(
                range_<3,5>(
                    utils::static_for<0, 5, 1, add_loop_outer>(
                        tag<0>(
                            all(LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(
                                range<3,5>(init())))
                              ), 
                        tag<0>()
                    )
                )
            );

QDevice<QDeviceType::qiskit_qasm_simulator, 6> qiskit;
std::cout << qiskit(expr) << std::endl;

OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
creg c[6];
h q[3];
cu1(1.570796326794896558) q[4], q[3];
cu1(0.785398163397448279) q[5], q[3];
h q[4];
cu1(1.570796326794896558) q[5], q[4];
h q[5];
cu1(3.141592653589793116) q[0], q[3];
cu1(1.570796326794896558) q[1], q[3];
cu1(0.785398163397448279) q[2], q[3];
cu1(1.570796326794896558) q[1], q[4];
cu1(0.785398163397448279) q[2], q[4];
cu1(0.785398163397448279) q[2], q[5];
h q[3];
cu1(1.570796326794896558) q[4], q[3];
cu1(0.785398163397448279) q[5], q[3];
h q[4];
cu1(1.570796326794896558) q[5], q[4];
h q[5];

